In [19]:
# Note: In the result file the title of the 'EVAoutput' section
# is either as 'EVAoutput' or 'EVAOutput' so the script needs
# to be changed based on the heading 
# (Usually if the first letter of the timeseries name is Captital, 
# the heading is 'EVAOutput' and vice-versa)


import mikeio
import os
import glob
import pandas as pd
import numpy as np

# Setting the excel output file path and name
output_file_name = 'Storm Surge Statistical Analysis'
output_directory = r'G:\MJI\SKZ\Script\Strom Surge stastical Analysis_Trial\Eva'

# Creating the output file in the folder path
output_file_path =os.path.join(output_directory, '{}.xlsx'.format(output_file_name))
writer = pd.ExcelWriter(output_file_path, engine='openpyxl')

# Iterating through all the eva result file in the folder
# to extract the required eva analysis result
for file_name in glob.glob(r"G:\MJI\SKZ\Script\Strom Surge stastical Analysis_Trial\Result\*.res"):
    
    # Splitting the path list to create a file path list
    file_path_list = file_name.split(os.sep)
    
    # Reading the mike eva analysis result pfs file
    pfs = mikeio.read_pfs(file_name)

    # Creating the list of probability distribution method used for analysis
    method_list = list(pfs.EVAOutput.DistributionAnalysis.keys())[6:]

    # Defining a blank dataframe to store the analysis result information
    analysis_df = pd.DataFrame()

    # Creating a list from the return period used for analysis
    return_period_list = pfs.EVAOutput.DistributionAnalysis.ReturnPeriods

    # Iterating through each probability method to extract the required analysis information
    for item_name in method_list:

        # Defining a dataframe to store particular method analysis information
        method_df = pd.DataFrame()

        # Extracting the distribution method label/name used in the reult file
        method_name = pfs.EVAOutput.DistributionAnalysis[item_name].ItemName

        # Creating a list from the all the goodness parameter that is available in the mike eva analysis tool
        goodness_fit_parameter_list = list(pfs.EVAOutput.DistributionAnalysis[item_name].GoodnessOfFitStatistics.keys())

        # Extacting the estimated, average quantile distribution value
        # and also the standard deviation value
        estimated_quantile = pfs.EVAOutput.DistributionAnalysis[item_name].TyearEventTable.TyearEvent
        average_quantile = pfs.EVAOutput.DistributionAnalysis[item_name].TyearEventTable.AvgTyearEvent
        standard_deviation = pfs.EVAOutput.DistributionAnalysis[item_name].TyearEventTable.StdTyearEvent

        # Creating individual dataframe using the average, estimate quantile data and standard deviation data
        estimated_df = pd.DataFrame({'Item type': 'Estimated Quantile', 'Return Period (years)': return_period_list,
                                    method_name : estimated_quantile})

        average_df = pd.DataFrame({'Item type': 'Average Quantile', 'Return Period (years)': return_period_list,
                                    method_name : average_quantile})

        standard_deviation_df = pd.DataFrame({'Item type': 'Standard deviation', 'Return Period (years)': return_period_list,
                                    method_name : standard_deviation})

        # Defining a list to store the goodness test name
        goodness_test_list = []

        # Iterating through all the goodness parameter and
        # Extracting only the goodness test name
        for parameter in goodness_fit_parameter_list:

            if parameter[-4:] == 'Test':
                goodness_test_list.append(parameter)

            else:
                pass


        # Defining blank list to store the goodness test parameter 
        # and analysis goodness test value 
        goodness_parameter = []
        goodness_statistics = []


        # Iterating through the goodness fit test list and checking whether it is used or not
        # if it retruns "True" then appending the test name and calculated value into the list
        for test_type in goodness_test_list:

            if pfs.EVAOutput.DistributionAnalysis[item_name].GoodnessOfFitStatistics[test_type] == True:

                stat_item_name = '{}Statistic'.format(test_type[:-4])
                test_type_statistic = pfs.EVAOutput.DistributionAnalysis[item_name].GoodnessOfFitStatistics[stat_item_name]

                goodness_parameter.append(test_type)
                goodness_statistics.append(test_type_statistic)

            else:
                pass


        # Creating dataframe using the of goodness fit test information
        goodness_df = pd.DataFrame({'Item type': 'Goodness of fit', 'Return Period (years)': goodness_parameter,
                                    method_name : goodness_statistics})

        # Updating the estimated quantile, average quantile, standard deviation and goodness fit test
        # Information into the distribution method dataframe
        method_df = pd.concat([estimated_df, average_df, standard_deviation_df, goodness_df])


        # Updating the distribution method information into the combined method dataframe
        if analysis_df.empty:
            analysis_df = pd.concat([analysis_df, method_df])

        else:
            analysis_df = pd.merge(analysis_df, method_df, on = ['Item type', 'Return Period (years)'])
    
    # Adding the analysis summary as a new workbook in the excel file
    analysis_df.to_excel(writer, sheet_name= "{}".format(file_path_list[-1][:-4]), index=False)

# Saving the excel file
writer.save()